We start by adding the necessary folders to the current working path.

In [ ]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-4])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# add TRA folder to current working path
sys.path.append(parent_directory+"/demos/AdvectiveBarriers/TRA3D")

# add ActiveBarriers folder to current working path
sys.path.append(parent_directory+"/demos/ActiveBarriers")

# Overview

We illustrate the numerical implementation and the use of the active Trajectory Rotation Average ($ \mathrm{a\overline{TRA}} $) on a directionally steady Beltrami flow such as the 3D viscous, unsteady version of the classic ABC-flow. The active LCS diagnostic is applied to the three-dimensional and steady instantaneous linear momentum/vorticity barrier equation. The notebook is structured as follows:

1. Import vorticity and velocity data stored in the folder 'data/ABC'.
<br />
2. Define computational parameters (such as the number of cores) and data.
 <br />
3. Define spatio-temporal domain.
 <br />
4. Interpolate velocity from (discrete) gridded data.
 <br />
5. Apply active Trajectory Rotation Average ($ \mathrm{a\overline{TRA}} $) to instantaneous linear momentum barrier equation:

   For 3D incompressible, uniform-density Navier–Stokes flows, the Eulerian barrier equation for linear momentum is:
   \begin{equation}
   \mathbf{x'}(s) = \nu \rho \Delta \mathbf{v}(\mathbf{x}, t) \label{eq: linearmoment1} \tag{1}
   \end{equation}
   
   For 3D incompressible, uniform-density Navier–Stokes flows, the Eulerian barrier equation for vorticity is:

   \begin{equation}
   \mathbf{x'}(s) = \nu \rho \Delta \mathbf{\omega}(\mathbf{x}, t) \label{eq: vorticity1} \tag{2}
   \end{equation}
    
   It can be verified that the 3D viscous, unsteady version of the classic ABC-flow satisfies the strong Beltrami property:
   
   \begin{equation}
   \mathbf{\omega}(\mathbf{x},t) = \mathbf{v}(\mathbf{x}, t) \label{eq: Beltrami} \tag{3}
   \end{equation}
   
   By definition it follows:

   \begin{equation}
   \mathbf{\Delta v}(\mathbf{x},t) = \mathbf{\Delta \omega}(\mathbf{x},t) = \mathbf{\nabla}(\mathbf{ \nabla \omega})-\mathbf{\nabla} \times (\mathbf{\nabla \times \mathbf{\omega}}) = \mathbf{v}(\mathbf{x}, t) \label{eq: LaplacianOmega} \tag{4}
   \end{equation}
   
   The Eulerian barrier equation for both linear momentum/vorticity reduce to:
   <br />
   \begin{equation}
   \mathbf{x'}(s) = -\nu \rho \mathbf{v}(\mathbf{x}, t) \label{eq: linearmoment2} \tag{5}
   \end{equation}
   
   By appropriate rescaling with respect to the dummy time, equation \ref{eq: linearmoment2} simplifies to: 
   
   \begin{equation}
   \mathbf{x'}(s) = \Delta \mathbf{v}(\mathbf{x}, t) = \Delta \mathbf{\omega}(\mathbf{x}, t) = \mathbf{v}(\mathbf{x}, t) \label{eq: linearmoment3} \tag{6}
   \end{equation}
   
    In order to compute \ref{eq: linearmoment3}, we numerically evaluate the laplacian of the velocity field. Generally the strong Beltrami property of the flow is not know, whenever the data is gathered from numerical/experimental data and hence one needs to compute $ \Delta \mathbf{v}(\mathbf{x}, t) $ in order to evaluate Eulerian active barriers.
   
    Time 't' is just a fixed parameter and hence the system is autonomous and the integration is done with respect to a dummy time variable 's'. 's' can be arbitrarily large and the $ \mathrm{a\overline{TRA}} $ is only limited by the spatial resolution and domain. It can thus potentially reveal finer and more resolved structures compared to the classic version of the $ \mathrm{\overline{TRA}} $, which is fundamentally limited by the temporal domain of the available data.
    
   We start by evaluating the right-hand-side of equation \ref{eq: linearmoment3} by first interpolating $ \mathbf{v}(\mathbf{x}, t) $ over the spatial meshgrid.
   
   We then apply the $ \mathrm{a\overline{TRA}} $ to the autonomous ODE from equation \ref{eq: linearmoment3} over different dummy time-intervals.

6. References

# Import data

In [ ]:
# import scipy
import scipy.io as sio

# Import velocity data from file in data-folder
mat_file = sio.loadmat('../../../../data/ABC/ABCunsteady.mat')

U = mat_file['u'] # array (NY, NX, NZ)
V = mat_file['v'] # array (NY, NX, NZ)
W = mat_file['w'] # array (NY, NX, NZ)
x = mat_file['x'] # array (1, NX)
y = mat_file['y'] # array (1, NY)
z = mat_file['z'] # array (1, NZ)
time_data = mat_file['t']

# Computational parameters and data

Here we define the computational parameters and the data.

In [ ]:
# import numpy
import numpy as np

# Number of cores for parallel computing
Ncores = 7 # int

# Periodic boundary conditions
periodic_x = True # bool
periodic_y = True # bool
periodic_z = True # bool
periodic = [periodic_x, periodic_y, periodic_z]

# Unsteady velocity field
bool_unsteady = True # bool

# Defined domain
defined_domain = np.isfinite(U).astype(int) # array (NY, NX, NZ)

## Compute meshgrid of dataset
X, Y, Z = np.meshgrid(x, y, z) # array (NY, NX), array (NY, NX, NZ)

## Resolution of meshgrid
dx_data = X[0,1,0]-X[0,0,0] # float
dy_data = Y[1,0,0]-Y[0,0,0] # float
dz_data = Y[0,0,1]-Z[0,0,0] # float

delta = [dx_data, dy_data, dx_data] # list (3, )

# Spatio-temporal domain

Here we define the spatio-temporal domain over which to consider the dynamical system.

In [ ]:
# Initial time
t0 = 0 # float

# boundaries
xmin = 0 # float
xmax = 2*np.pi # float
ymin = 0 # float
ymax = 2*np.pi # float
zmin = 0 # float
zmax = 2*np.pi # float

# Resolution of meshgrid
Ny = 200 # int
Nx = 200 # int
Nz = 200 # int

x_domain = np.linspace(xmin, xmax, Nx, endpoint = True) # array (Nx, )
y_domain = np.linspace(ymin, ymax, Ny, endpoint = True) # array (Ny, )
z_domain = np.linspace(zmin, zmax, Nz, endpoint = True) # array (Nz, )

dx = x_domain[1]-x_domain[0] # float
dy = y_domain[1]-y_domain[0] # float
dz = z_domain[1]-z_domain[0] # float

X_domain, Y_domain, Z_domain = np.meshgrid(x_domain, y_domain, z_domain) # array (Ny, Nx, Nz)

# Define dummy time-intervals for computation of aTRA in ascending order
s1 = 10 # float s1 > 0
s2 = 20 # float s2 > s1
s3 = 30 # float s3 > s2

s = [s1, s2, s3] # list (3,)

# define dummy time interval
ds = 0.1
sN = s[-1] # Feel free to change this parameter as needed
s_dummy = np.arange(0, sN+ds, ds)

# Interpolate velocity and Laplacian of velocity

In order to evaluate the velocity field at arbitrary locations, we interpolate the discrete velocity data. The interpolation with respect to time and space is "linear".
We further compute the laplacian of the velocity field and construct an interpolation object for the laplacian of the velocity field.

In [ ]:
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_unsteady, interpolant_steady

# Import function to compute laplacian of velocity from structured velocity data.
from ipynb.fs.defs.laplacian_velocity import laplacian_velocity

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_unsteady(X, Y, Z, U, V, W, time_data)

Interpolant_u = Interpolant[0] # RectangularBivariateSpline-object
Interpolant_v = Interpolant[1] # RectangularBivariateSpline-object
Interpolant_w = Interpolant[2] # RectangularBivariateSpline-object

# Compute laplacian of velocity
Lap_u, Lap_v, Lap_w  = laplacian_velocity(t0, X, Y, Z, U, V, W, periodic, bool_unsteady, 0, time_data)

# Set nan values to zero (in case there are any) so that we can apply interpolant. 
# Interpolant does not work if the array contains nan values. 
Lap_u[np.isnan(Lap_u)] = 0
Lap_v[np.isnan(Lap_v)] = 0
Lap_w[np.isnan(Lap_w)] = 0

# Interpolate laplacian of velocity data using cubic spatial interpolation
Interpolant_lap = interpolant_steady(X, Y, Z, Lap_u, Lap_v, Lap_w)

Interpolant_lap_u = Interpolant_lap[0] # RectangularBivariateSpline-object
Interpolant_lap_v = Interpolant_lap[1] # RectangularBivariateSpline-object
Interpolant_lap_w = Interpolant_lap[2] # RectangularBivariateSpline-object

# Instantaneous linear momentum/vorticity barriers from active Trajectory Rotation Average ($ \mathrm{a\overline{TRA}} $)

We visualize linear momentum/vorticity barriers by applying the $ \mathrm{a\overline{TRA}} $ to trajectories of instantaneous linear momentum/vorticity barrier equation \ref{eq: linearmoment3}. As soon as the $ \mathrm{a\overline{TRA}} $-diagnostic is applied to an active vector field, such as linear momentum (or vorticity), we refer to it as $ \mathrm{a\overline{TRA}} $ (active $ \mathrm{\overline{TRA}} $).

We now compute the active $ \mathrm{a\overline{TRA}} $ for the instantaneous linear momentum/vorticity barrier equations. First, we evaluate the velocity along the particle trajectories from the autonomous ODE (eq. \ref{eq: linearmoment2}). From the velocity of the trajectories (=time-derivative of flow map) we then compute the $ \mathrm{\overline{TRA}} $.The computation of the active $ \mathrm{a\overline{TRA}} $ is not done over the 3D meshgrid but only over the 2D faces of the cube $ [0,2\pi]^3 $.

In [ ]:
# Import function to compute trajectory/velocity along trajectory
from ipynb.fs.defs.integration_dFdt import integration_dFdt

# Import function to compute trajectory rotation average (TRA)
from ipynb.fs.defs.TRA import _TRA

# Import package for parallel computing
from joblib import Parallel, delayed

def compute_aTRA(x0, y0, z0, s_dummy, s):
    
    X0 = np.array([x0, y0, z0]) # array (3, Nx*Ny*Nz)
    
    DFdt = integration_dFdt(s_dummy, X0, X, Y, Z, Interpolant_lap_u, Interpolant_lap_v, Interpolant_lap_w, periodic, False)[1] # array (Nt, 3, Nx*Ny*Nz)
    
    aTRA = np.zeros((len(s), DFdt.shape[-1]))
    
    counter = 0
    for idx in np.searchsorted(s_dummy, s):
        
        for i in range(DFdt.shape[-1]):
            
            aTRA[counter, i] = _TRA(DFdt[:idx+1,:,i].transpose(), s_dummy[idx])
            
        counter += 1
    
    return aTRA

# Split x0,y0,z0 into 'Ncores' equal batches for parallel computing
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

## XY-plane

Compute $ \mathrm{a\overline{TRA}} $ on XY-plane.

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny*Nz, )
x0_xy = X_domain[:,:,-1].ravel() # array (Nx*Ny, )
y0_xy = Y_domain[:,:,-1].ravel() # array (Nx*Ny, )
z0_xy = Z_domain[:,:,-1].ravel() # array (Nx*Ny, )

x0_batch = list(split(x0_xy, Ncores)) # list (Nx*Ny)
y0_batch = list(split(y0_xy, Ncores)) # list (Nx*Ny)
z0_batch = list(split(z0_xy, Ncores)) # list (Nx*Ny)
results = Parallel(n_jobs=Ncores, verbose = 2)(delayed(compute_aTRA)(x0_batch[i], y0_batch[i], z0_batch[i], s_dummy, s) for i in range(len(x0_batch)))

# Extract results of aTRA computation on xy-plane
aTRA_xy = results[0]
for res in results[1:]:
    aTRA_xy = np.append(aTRA_xy, res, axis = -1)
    
X0_xy = np.array(x0_xy).reshape(Ny,Nx) # array (Ny, Nx)
Y0_xy = np.array(y0_xy).reshape(Ny,Nx) # array (Ny, Nx)
Z0_xy = np.array(z0_xy).reshape(Ny,Nx) # array (Ny, Nx)
aTRA_xy = np.array(aTRA_xy).reshape(len(s),Ny,Nx) # array (3, Ny, Nx)

## XZ-plane

Compute $ \mathrm{a\overline{TRA}} $ on XZ-plane.

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny*Nz, )
x0_xz = X_domain[0,:,:].ravel() # array (Nx*Nz, )
y0_xz = Y_domain[0,:,:].ravel() # array (Nx*Nz, )
z0_xz = Z_domain[0,:,:].ravel() # array (Nx*Nz, )

x0_batch = list(split(x0_xz, Ncores)) # list (Nx*Nz)
y0_batch = list(split(y0_xz, Ncores)) # list (Nx*Nz)
z0_batch = list(split(z0_xz, Ncores)) # list (Nx*Nz)
results = Parallel(n_jobs=Ncores, verbose = 2)(delayed(compute_aTRA)(x0_batch[i], y0_batch[i], z0_batch[i], s_dummy, s) for i in range(len(x0_batch)))

# Extract results of aTRA computation on xz-plane
aTRA_xz = results[0]
for res in results[1:]:
    aTRA_xz = np.append(aTRA_xz, res, axis = -1)
    
X0_xz = np.array(x0_xz).reshape(Nz,Nx) # array (Ny, Nx)
Y0_xz = np.array(y0_xz).reshape(Nz,Nx) # array (Ny, Nx)
Z0_xz = np.array(z0_xz).reshape(Nz,Nx) # array (Ny, Nx)
aTRA_xz = np.array(aTRA_xz).reshape(len(s),Nz,Nx) # array (3, Ny, Nx)

## YZ-plane

Compute $ \mathrm{a\overline{TRA}} $ on YZ-plane.

In [ ]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny*Nz, )
x0_yz = X_domain[:,0,:].ravel() # array (Ny*Nz, )
y0_yz = Y_domain[:,0,:].ravel() # array (Ny*Nz, )
z0_yz = Z_domain[:,0,:].ravel() # array (Ny*Nz, )

x0_batch = list(split(x0_yz, Ncores)) # list (Ny*Nz)
y0_batch = list(split(y0_yz, Ncores)) # list (Ny*Nz)
z0_batch = list(split(z0_yz, Ncores)) # list (Ny*Nz)
results = Parallel(n_jobs=Ncores, verbose = 2)(delayed(compute_aTRA)(x0_batch[i], y0_batch[i], z0_batch[i], s_dummy, s) for i in range(len(x0_batch)))

# Extract results of aTRA computation on xz-plane
aTRA_yz = results[0]
for res in results[1:]:
    aTRA_yz = np.append(aTRA_yz, res, axis = -1)
    
X0_yz = np.array(x0_yz).reshape(Nz,Ny) # array (Nz, Ny) 
Y0_yz = np.array(y0_yz).reshape(Nz,Ny) # array (Nz, Ny) 
Z0_yz = np.array(z0_yz).reshape(Nz,Ny) # array (Nz, Ny) 
aTRA_yz = np.array(aTRA_yz).reshape(len(s),Nz,Ny) # array (3, Nz, Ny)

##### Visualize aTRA over different dummy time-intervals

In the following we visualize the instantaneous linear momentum/vorticity barriers at time $ t_0 = 0 $ through the $ \mathrm{a\overline{TRA}} $ over different dummy time-intervals. Ridges of $ \mathrm{a\overline{TRA}}_{0,0}^{s_i} \ (i \in \lbrace 1,2,3 \rbrace) $ indicate transport barriers which block the transport of linear momentum. Note that contrary to the classic $ \mathrm{\overline{TRA}} $, the $ \mathrm{a\overline{TRA}} $ diagnostic is not limited by the temporal domain as the dummy time-interval can be chosen to be arbitrarily large. We explore the structures of the $ \mathrm{a\overline{TRA}}_{0,t_0}^{s_i} $ over different dummy time-intervals $ s_i \in \lbrace 1,2,3 \rbrace $ at time $ t_0 $.

In [ ]:
############################################ PLOT RESULTS ############################################

# Import plotly for 3D figures
from plotly import graph_objs as go

# iterate over different dummy time-intervals and display aTRA field calculated over each time-interval
def plot_aTRA(i):

    # define minimum and maximum values for colorbar
    min_value = min(np.min(aTRA_yz[i,:,:]),min(np.min(aTRA_xy[i,:,:]), np.min(aTRA_xz[i,:,:])))
    max_value = max(np.max(aTRA_yz[i,:,:]),max(np.max(aTRA_xy[i,:,:]), np.max(aTRA_xz[i,:,:])))

    # create figure
    fig = go.Figure(data=[go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy, surfacecolor = aTRA_xy[i,:,:], showscale = True, colorscale='rainbow', cmin=min_value, cmax=max_value)])
    fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz, z = Z0_xz, surfacecolor = aTRA_xz[i,:,:], showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
    fig.add_trace(go.Surface(x = X0_yz, y = Y0_yz, z = Z0_yz, surfacecolor = aTRA_yz[i,:,:], showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
    fig.add_trace(go.Surface(x = X0_xy, y = Y0_xy, z = Z0_xy-2*np.pi, surfacecolor = aTRA_xy[i,:,:], showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
    fig.add_trace(go.Surface(x = X0_xz, y = Y0_xz+2*np.pi, z = Z0_xz, surfacecolor = aTRA_xz[i,:,:], showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))
    fig.add_trace(go.Surface(x = X0_yz+2*np.pi, y = Y0_yz, z = Z0_yz, surfacecolor = aTRA_yz[i,:,:], showscale = False, colorscale='rainbow', cmin=min_value, cmax=max_value))

    # camera
    camera = dict(eye=dict(x=-1.45,y=-1.45,z=1.7))
    
    fig.update_layout(scene_camera=camera)
    
    fig.update_layout(
    title={
        'text': f'$\mathrm{{\overline{{aTRA}}}}_{{{0,t0}}}^{{{s[i]}}}$',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

    fig.show()

## $ \mathrm{a\overline{TRA}}_{0,t_0}^{s_1} $

In the following we visualize the instantaneous linear momentum barriers/vorticity at time $ t_0 = 0 $ through $ \mathrm{a\overline{TRA}}_{0,t_0}^{s_1} $.

In [ ]:
############################################ PLOT RESULTS ############################################

plot_aTRA(0)

## $ \mathrm{a\overline{TRA}}_{0,t_0}^{s_2} $

In the following we visualize the instantaneous linear momentum/vorticity barriers at time $ t_0 = 0 $ through $ \mathrm{a\overline{TRA}}_{0,t_0}^{s_2} $.

In [ ]:
############################################ PLOT RESULTS ############################################

plot_aTRA(1)

## $ \mathrm{a\overline{TRA}}_{0,t_0}^{s_3} $

In the following we visualize the instantaneous linear momentum/vorticity barriers at time $ t_0 = 0 $ through $ \mathrm{a\overline{TRA}}_{0,t_0}^{s_3} $.

In [ ]:
############################################ PLOT RESULTS ############################################

plot_aTRA(2)

The figures focus on momentum/vorticity-based active elliptic features of the 3D viscous, unsteady ABC-flow. 
The $ \mathrm{a\overline{TRA}} $ provides a clear picture of the main momentum/vorticity trapping vortices. The features become even more pronounced for longer s-times. The equivalence between instantaneous momentum and vorticity barriers holds as the ABC-flows satisfies the strong Beltrami property.

# References

[1] Haller, G., Katsanoulis, S., Holzner, M., Frohnapfel, B., & Gatti, D. (2020). Objective barriers to the transport of dynamically active vector fields. Journal of Fluid Mechanics, 905.

[2] Haller, G., Aksamit, N., & Encinas-Bartos, A. P. (2021). Quasi-objective coherent structure diagnostics from single trajectories. Chaos: An Interdisciplinary Journal of Nonlinear Science, 31(4), 043131.

[3] Notebook 9.9. in "Transport Barriers and Coherent Structures in Flow Data" by Prof. George Haller.

[4] [YouTube Tutorial](https://youtu.be/gDl-ybKULgE)